# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [70]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

input_file = os.path.join('Output Data', 'cities.csv')

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
weather_df = pd.read_csv(input_file)
weather_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,punta arenas,20,CL,1615395030,46,-53.1500,-70.9167,51.80,31.07
1,1,saint-philippe,0,RE,1615395036,74,-21.3585,55.7679,80.60,16.11
2,2,kandrian,99,PG,1615395039,85,-6.2167,149.5500,77.31,3.60
3,3,cidreira,0,BR,1615395041,70,-30.1811,-50.2056,75.00,9.35
4,4,mosquera,40,CO,1615395043,52,4.7059,-74.2302,66.20,2.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
gmaps.configure(api_key=g_key)

In [34]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"]

In [35]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations,weights = humidity,dissipating=False,max_intensity=200, 
                                 point_radius = 5)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [117]:
weather_new_df = weather_df.loc[(weather_df['Max Temperature'] < 80) &
                               (weather_df['Max Temperature'] > 70) &
                               (weather_df['Wind Speed'] < 8) &
                               (weather_df['Cloudiness'] == 0)]

weather_new_df = weather_new_df.dropna()
hotel_df = weather_new_df.reset_index(drop = True)
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,39,ilo,0,PE,1615394845,69,-17.6394,-71.3375,77.00,5.75
1,276,kutum,0,SD,1615395753,15,14.2000,24.6667,78.04,7.00
2,277,haifa,0,IL,1615395755,46,32.8156,34.9892,73.40,5.75
3,284,umm lajj,0,SA,1615395773,52,25.0213,37.2685,78.91,3.76
4,306,san luis de la paz,0,MX,1615395827,35,21.3000,-100.5167,73.00,0.58
5,307,fatehpur,0,IN,1615395829,19,25.9333,80.8000,76.35,6.22
6,321,murud,0,IN,1615395862,56,18.3258,72.9650,77.09,4.03
7,338,jhanjharpur,0,IN,1615395903,28,26.2667,86.2833,73.58,4.45
8,510,gaya,0,IN,1615396351,68,24.7833,85.0000,73.40,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [118]:
hotel_df["Hotel name"] = ''
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel name
0,39,ilo,0,PE,1615394845,69,-17.6394,-71.3375,77.00,5.75,
1,276,kutum,0,SD,1615395753,15,14.2000,24.6667,78.04,7.00,
2,277,haifa,0,IL,1615395755,46,32.8156,34.9892,73.40,5.75,
3,284,umm lajj,0,SA,1615395773,52,25.0213,37.2685,78.91,3.76,
4,306,san luis de la paz,0,MX,1615395827,35,21.3000,-100.5167,73.00,0.58,
5,307,fatehpur,0,IN,1615395829,19,25.9333,80.8000,76.35,6.22,
6,321,murud,0,IN,1615395862,56,18.3258,72.9650,77.09,4.03,
7,338,jhanjharpur,0,IN,1615395903,28,26.2667,86.2833,73.58,4.45,
8,510,gaya,0,IN,1615396351,68,24.7833,85.0000,73.40,4.61,


In [114]:
target_type = "lodging"
lat = 24.7833
lon = 85.0000
#radius = 10000
#coord = f"{lat},{lon}"
coord = "24.7833,85.0000"
name = 'gaya'
# rewrite params dict

params = {
   # "keyword":name,
    "location": coord,
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

response = requests.get(base_url, params = params).json()
pprint(response)
#result = response['results']

#print(result[0]['name'])

#print(response['results'][0]['name'])

{'html_attributions': [],
 'next_page_token': 'ATtYBwL8bhMqfeSpnukDHsVacDfC_E5W2LppttoNZtxaQ5Wg6WMq6tch9B-BfEPHZC8rc2BCbWb4vx2UeH1ehjHHTRjoG2rb1sWDLtouA5t2k5S7Yjl8RC12hGcY2AGq5rke8nM4ARVcajIN8dTKirXXpdqnTJg2-xJNeG_M4PTs7Fg5cmeO-Y3n_S1T9USOGB43Oz13BqHSi2J6uMWari0V49_tc0foFmCGvdsk3PXW6r1GlcrQNgbVYJBAXpCx-QQPU_CUDp3HhzjvpE5p7SHb8SQVKcDQ9-a6YyHuhA7l25uQoxJ05Z4nl9RlLLxuNVwYJIeAjpxrC-gWJOGAB-Lfk6aYMcqbUp4Fc1p-P2z9XgHpSM_KQw1PHbrgZAzxfvWtFMM26316oanppmh0DC2SsJwBmirAjlppAr_bXCdWrhzfQqDCTmJYfiFX',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 24.7779164, 'lng': 85.0094259},
                           'viewport': {'northeast': {'lat': 24.7792969302915,
                                                      'lng': 85.0106166802915},
                                        'southwest': {'lat': 24.7765989697085,
                                                      'lng': 85.0079187197085}}},
              'icon': 'https://maps.gstatic.com/mapfiles/pl

              'rating': 3.1,
              'reference': 'ChIJnSXckD0q8zkRlpP74ebQoDs',
              'scope': 'GOOGLE',
              'types': ['lodging', 'point_of_interest', 'establishment'],
              'user_ratings_total': 16,
              'vicinity': 'Chowk, KP Road, Dulhingunj, Gaya'},
             {'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 24.7991858, 'lng': 85.0007546},
                           'viewport': {'northeast': {'lat': 24.8005294302915,
                                                      'lng': 85.0021658802915},
                                        'southwest': {'lat': 24.79783146970849,
                                                      'lng': 84.9994679197085}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'name': 'Hotel Skylark',
              'photos': [{'height': 2304,
                          'html_attributions': ['<a '
                    

In [119]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000

params = {
    "radius": radius,
    "type": "lodging",
     "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    city_name = row['City']
    lat = row['Latitude']
    lng = row['Longitude']
    coord = f"{lat},{lng}"
    params['location'] = coord 
   
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel name'] = results[0]['name']
        print(response['results'][0]['name'])

    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel name'] = ''
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: ilo.
Closest hotel to ilo is Torreblanca Hotel.
Torreblanca Hotel
------------
Retrieving Results for Index 1: kutum.
Closest hotel to kutum is Dibbo areaمنطقة دبو.
Dibbo areaمنطقة دبو
------------
Retrieving Results for Index 2: haifa.
Closest hotel to haifa is Dan Panorama Haifa Hotel.
Dan Panorama Haifa Hotel
------------
Retrieving Results for Index 3: umm lajj.
Closest hotel to umm lajj is Moon light Furnished Units.
Moon light Furnished Units
------------
Retrieving Results for Index 4: san luis de la paz.
Closest hotel to san luis de la paz is Hotel Real San Luis.
Hotel Real San Luis
------------
Retrieving Results for Index 5: fatehpur.
Closest hotel to fatehpur is Hotel Bombay Lodge.
Hotel Bombay Lodge
------------
Retrieving Results for Index 6: murud.
Closest hotel to murud is On Point Plaza.
On Point Plaza
------------
Retrieving Results for Index 7: jhanjharpur.
Closest hotel to jhanjharpur is Krishna Sweets.
Krishna Sweets
------------
Retr

In [120]:
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel name
0,39,ilo,0,PE,1615394845,69,-17.6394,-71.3375,77.00,5.75,Torreblanca Hotel
1,276,kutum,0,SD,1615395753,15,14.2000,24.6667,78.04,7.00,Dibbo areaمنطقة دبو
2,277,haifa,0,IL,1615395755,46,32.8156,34.9892,73.40,5.75,Dan Panorama Haifa Hotel
3,284,umm lajj,0,SA,1615395773,52,25.0213,37.2685,78.91,3.76,Moon light Furnished Units
4,306,san luis de la paz,0,MX,1615395827,35,21.3000,-100.5167,73.00,0.58,Hotel Real San Luis
5,307,fatehpur,0,IN,1615395829,19,25.9333,80.8000,76.35,6.22,Hotel Bombay Lodge
6,321,murud,0,IN,1615395862,56,18.3258,72.9650,77.09,4.03,On Point Plaza
7,338,jhanjharpur,0,IN,1615395903,28,26.2667,86.2833,73.58,4.45,Krishna Sweets
8,510,gaya,0,IN,1615396351,68,24.7833,85.0000,73.40,4.61,Hotel Kumar Kripa


In [123]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [124]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [126]:
fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))